In [3]:
import requests
import os
import uuid
import re
base_url = 'http://localhost:8000/api/v0.0'


In [23]:
ngsi_msg = {
  "headers": [
    {
      "fiware-service": "tdm"
    },
    {
      "fiware-servicePath": "/cagliari/edge/meteo"
    },
    {
      "timestamp": 1531774294021
    }
  ],
  "body": {
    "attributes": [
      {
        "name": "altitude",
        "type": "float",
        "value": " "
      },
      {
        "name": "dateObserved",
        "type": "String",
        "value": "2018-07-16T20:51:33+00:00"
      },
      {
        "name": "location",
        "type": "geo:point",
        "value": "39.2479168, 9.1329701"
      },
      {
        "name": "timestamp",
        "type": "Integer",
        "value": "1531774293"
      },
      {
        "name": "windDirection",
        "type": "Float",
        "value": "174.545"
      },
      {
        "name": "windSpeed",
        "type": "Float",
        "value": "1.000"
      },
      {
        "name": "latitude",
        "type": "string",
        "value": "39.2479168"
      },
      {
        "name": "longitude",
        "type": "string",
        "value": "9.1329701"
      }
    ],
    "type": "WeatherObserved",
    "isPattern": "false",
    "id": "WeatherObserved:Edge-CFA703F4.esp8266-7806085.Davis"
  }
}




In [24]:
p = re.compile(r'(?P<Type>)\w+:(?P<Edge>[a-zA-Z0-9_-]+)\.(?P<Node>[a-zA-Z0-9_-]+)\.(?P<Sensor>[a-zA-Z0-9_-]+)')
sensor_type_type, edge, node_name, sensor_type_name = p.search(ngsi_msg['body']['id']).groups()
sensor_name = '{}.{}'.format(node_name, sensor_type_name)




In [25]:
def get_geometry(attributes):
    geom = {}
    for attr in attributes:
        if attr['name'] in {'latitude', 'longitude'}:
            geom[attr['name']] = float(attr['value'])
    return [geom['latitude'], geom['longitude']]
        

In [26]:
geometry = {"type": "Point", "coordinates": get_geometry(ngsi_msg['body']['attributes'])}

In [27]:
non_properties = {'latitude', 'longitude', 'timestamp', 'dateObserved', 'location'}
properties = [ attr['name'] for attr in ngsi_msg['body']['attributes'] if attr['name'] not in non_properties]
sensor_type = {
        "code": str(uuid.uuid4()),
        "type": sensor_type_type,
        "name": sensor_type_name,
        "category": ["sensor"],
        "function": ["sensing"],
        "controlledProperty": properties
    }

In [15]:
requests.post(os.path.join(base_url, 'sensor_types'), json=[sensor_type])

<Response [200]>

In [16]:
requests.post(os.path.join(base_url, 'nodes'), json=[node])

<Response [404]>

In [40]:
stypecode = requests.get(os.path.join(base_url, 'sensor_types'), params={'name': sensor_type_name}).json()[0]['code']

In [41]:
sensor = {
    'code': str(uuid.uuid4()),
    'stypecode': stypecode,
    'geometry': geometry,
    'nodecode': str(uuid.uuid4()),
    'name': sensor_name
   
}

In [42]:
requests.post(os.path.join(base_url, 'sensors'), json=[sensor])

<Response [200]>